In [1]:
import pandas as pd
df_train=pd.read_csv('./data/train.txt',sep='\t',names=['label','text'])
df_train.head()

,label,text
0,entertainment,台媒 预测 周 冬雨 金马奖 封后 ， 大气 的 倪妮 却 佳作 难 出
1,food,农村 就是 好 ， 能 吃 到 纯天然 无 添加 的 野生 蜂蜜 ， 营养 又 健康
2,fashion,14 款 知性美 装 ， 时尚 惊艳 搁浅 的 阳光 轻熟 的 优雅
3,history,火焰喷射器 1000 度 火焰 烧死 鬼子 4 连 拍
4,society,18 岁 青年 砍死 88 岁 老兵


In [2]:
df_dev=pd.read_csv('./data/dev.txt',sep='\t',names=['label','text'])
df_dev.head()

,label,text
0,history,跟 日本 刀 、 大马士革 剑 、 阿拉伯 弯刀 比 ， 中国 冷兵器 凭 啥 脱颖而出 ？
1,finance,世界 物 联网 大会 明日 在 京 召开 龙头股 启动 在 即
2,essay,短短 4 句 话 ， 度 了 无数 人
3,sports,郭晶晶 曾 撮合 吴敏霞 与 章子怡 前男友 ， 拒绝 豪门 平淡 才 是 真
4,entertainment,岁月 这 把 杀猪刀 怎么 就 对 周迅 格外 凶残 呢 ?


In [3]:
df_test=pd.read_csv('./data/test_with_label.word',sep='\t',names=['label','text'])
df_test.head()

,label,text
0,baby,生完 小孩 ， 公公 伺候 我 坐月子 ， 很 羞涩 很 感动
1,fashion,唐艺昕 与 陈伟霆 为 初秋 的 情侣 做出 了 典范 ， 看 他们 如何 穿 情侣 衣
2,entertainment,同学聚会 美女 被 嘲笑 是 剩女 ， 当超帅 老公 带 着 儿子 出场 ， 全场 沸腾 了
3,finance,中国 供给 侧 至少 存在 六大 问题
4,world,2.5 万英镑 可住 戴妃 闺房


In [4]:
#train_X=df_train.sample(frac=1).reset_index(drop=True)
train_X=df_train
train_Y=train_X['label']
train_X=train_X['text']
#dev_X=df_dev.sample(frac=1).reset_index(drop=True)
dev_X=df_dev
dev_Y=dev_X['label']
dev_X=dev_X['text']
#test_X=df_test.sample(frac=1).reset_index(drop=True)
test_X=df_test
test_Y=test_X['label']
test_X=test_X['text']

In [5]:
import csv
vocab=pd.read_csv('./data/vocab.txt',sep='\t',names=['word','times'],quoting=csv.QUOTE_NONE)
vocab.head()

,word,times
0,，,124634
1,的,94820
2,！,49051
3,？,37853
4,了,33303


In [6]:
def find_Chinese(text): #去除非汉字
    words=text.split(' ')
    Chinese_txt=''
    for word in words:
        if word>=u'\u4e00' and word<=u'\u9fa5':
            Chinese_txt+=word+' '
    return Chinese_txt

In [7]:
file1='./data/中文停用词表.txt'
file2='./data/哈工大停用词表.txt'
file3='./data/四川大学机器智能实验室停用词库.txt'
stopwords_set=set()
with open(file1,'r',encoding='utf-8') as f1,\
    open(file2,'r',encoding='utf-8') as f2,\
    open(file3,'r',encoding='utf-8') as f3:
    for word in f1.readlines():
        stopwords_set.add(word.strip())
    for word in f2.readlines():
        stopwords_set.add(word.strip())
    for word in f3.readlines():
        stopwords_set.add(word.strip())

def remove_stopwords(text): #去除停用词
    words=text.split(' ')
    Chinese_txt=''
    for word in words:
        if word not in stopwords_set:
            Chinese_txt+=word+' '
    return Chinese_txt

In [8]:
def clean(df): #文本清洗
    for i in range(len(df)):
        df[i]=find_Chinese(df[i])
        df[i]=remove_stopwords(df[i])
    return df

In [9]:
train_X=clean(train_X)
dev_X=clean(dev_X)
test_X=clean(test_X)

In [10]:
def remove_empty(df_X,df_Y):
    remove_list=[]
    for i in range(len(df_X)):
        if df_X[i]=='':
            remove_list.append(i)
    remove_list.reverse()
    for i in remove_list:
        del df_X[i]
        del df_Y[i]
    return df_X,df_Y

train_X,train_Y=remove_empty(train_X,train_Y)
dev_X,dev_Y=remove_empty(dev_X,dev_Y)
test_X,test_Y=remove_empty(test_X,test_Y)

In [11]:
from bert_serving.client import BertClient
bc=BertClient()
train_X_vec=bc.encode([x for x in train_X])
import numpy as np
np.savetxt('train_vec.txt',train_X_vec,fmt='%f',delimiter=' ')
np.savetxt('train_label.txt',train_Y,fmt='%s')
print('train_vec finished...')

D:\Anaconda\envs\Graduation\lib\site-packages\bert_serving\client\__init__.py:294: UserWarning: some of your sentences have more tokens than "max_seq_len=128" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  warnings.warn('some of your sentences have more tokens than "max_seq_len=%d" set on the server, '


train_vec finished...


In [12]:
dev_X_vec=bc.encode([x for x in dev_X])
np.savetxt('dev_vec.txt',dev_X_vec,fmt='%f',delimiter=' ')
np.savetxt('dev_label.txt',dev_Y,fmt='%s')
print('dev_vec finished...')

dev_vec finished...


In [13]:
test_X_vec=bc.encode([x for x in test_X])
np.savetxt('test_vec.txt',test_X_vec,fmt='%f',delimiter=' ')
np.savetxt('test_label.txt',test_Y,fmt='%s')
print('test_vec finished...')

test_vec finished...
